In [0]:
%sql
create catalog if not exists dev
managed location 'abfss://unity-catalog-storage@dbstoragembtc37rzuow5u.dfs.core.windows.net/1679904785127625'

In [0]:
%sql
create database if not exists dev.demodb


In [0]:
%sql
drop database if exists dev.default cascade

In [0]:
flight_time_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_dt = spark.read.format("json").schema(flight_time_ddl).\
        option("dateFormat", "M/d/y").\
        load('abfss://external-data@skstorage22062025.dfs.core.windows.net/flight-time/flight-time.json')


In [0]:
display(flight_time_dt)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.demodb.flight_time_ddl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
)
USING DELTA

LOAD DATA INTO DELTA TABLE

In [0]:
flight_time_dt.write.format("delta").mode("append").saveAsTable("dev.demodb.flight_time_ddl")

In [0]:
flight_time_tbl = spark.read.table("dev.demodb.flight_time_ddl")
display(flight_time_tbl)

CREATE A DELTA TABLE USING DELTATABLE API

In [0]:
(spark.read
    .format("json")
    .load("abfss://dbstorage@skstorage15062025.dfs.core.windows.net/external/flight-time.json")
    .display()
)



from delta import DeltaTable

(DeltaTable.createOrReplace(spark)
    .tableName("dev.demodb.flight_time_ddl")
    .addColumn("id", "INT")
    .addColumn("FL_DATE", "DATE")
    .addColumn("OP_CARRIER", "STRING")
    .addColumn("OP_CARRIER_FL_NUM", "INT")
    .addColumn("ORIGIN", "STRING")
    .addColumn("ORIGIN_CITY_NAME", "STRING")
    .addColumn("DEST", "STRING") 
    .addColumn("DEST_CITY_NAME", "STRING")
    .addColumn("CRS_DEP_TIME", "INT")
    .addColumn("DEP_TIME", "INT")
    .addColumn("WHEELS_ON", "INT")
    .addColumn("TAXI_IN", "INT")
    .addColumn("CRS_ARR_TIME", "INT")
    .addColumn("ARR_TIME", "INT")
    .addColumn("CANCELLED", "STRING")
    .addColumn("DISTANCE", "INT")
    .execute()
)

In [0]:
%sql
select * from dev.demodb.flight_time_ddl

SAVE DELTA DATA IN EXTERNAL LOCATION

In [0]:
flight_time = spark.read.table("dev.demodb.flight_time_ddl")
display(flight_time)

In [0]:
(flight_time.coalesce(1).write.format("delta").mode("overwrite").save("abfss://external-data@skstorage22062025.dfs.core.windows.net/flight-time/senthil"))

DELETE ONE RECORD FROM TBALE USING DELTATABLE API

In [0]:
people_df = spark.read.format("json").load("abfss://external-data@skstorage22062025.dfs.core.windows.net/flight-time/people.json")
display(people_df)



In [0]:
people_df.write.format("delta").mode("overwrite").saveAsTable("dev.demodb.people")
people = spark.read.table("dev.demodb.people")
display(people)

In [0]:
%sql
-- delete from dev.demodb.people where fname = 'abdul'

delete from dev.demodb.people where fname ='abdul';
    
select * from dev.demodb.people



In [0]:
%sql
Update dev.demodb.people set dob = '1985-05-25' where dob = '1975-05-25';
select * from dev.demodb.people

TIME TRAVEL

In [0]:
%sql
describe history dev.demodb.people

In [0]:
%sql
select * from dev.demodb.people version as of 4

In [0]:
%sql
select * from dev.demodb.people timestamp as of '2025-06-22T15:54:26.000+00:00'

DELETE THE DELTA TABLE DATA BY MISTAKE

In [0]:
%sql
drop table dev.demodb.people

In [0]:
%sql
select * from dev.demodb.people

ROLL BACK TO RESTORE

In [0]:
%sql 
describe history dev.demodb.people

In [0]:
%sql
restore table dev.demodb.people to timestamp as of '2025-06-22T16:00:43.000+00:00';
select * from dev.demodb.people